In [3]:
import pickle
from copy import deepcopy

from collections import defaultdict

anchors_file = "../datasets/anchors.pickle"
anchors_measurement_file = "../datasets/anchor_measurements_file.pickle"

with open(anchors_file, "rb") as f:
    anchors = pickle.load(f)

# load already existing measurements
try:
    with open(anchors_measurement_file, "rb") as f:
        anchor_measurements = pickle.load(f)
except FileNotFoundError:
    anchor_measurements = defaultdict(dict)

filtered_results = deepcopy(anchor_measurements)
print("retreived : ", len(anchor_measurements))
for i, anchor in enumerate(filtered_results):
    if len(anchor_measurements[anchor]['result']) == 1:
        if len(anchor_measurements[anchor]['result'][0]) == 0:
            anchor_measurements.pop(anchor)
            print("no results for anchor", anchor)
    else:
        print("more than one result")

    if anchor not in anchors:
        anchor_measurements.pop(anchor)
    # for index, node_results in enumerate(anchor_measurements[anchor]['result'][0]):
    #     if index > 10: break
    #     print(node_results)
    # print()

with open(anchors_measurement_file, "wb") as f:
    pickle.dump(anchor_measurements, f)

retreived :  784


## select probes with enough measurements

In [ ]:
with open(measurement_out_file, "rb") as f:
    atlas_measurement = pickle.load(f)

print(len(atlas_measurement))

filtered_measurements = {}
unchecked_probes = {}
for target_ip, measurements in atlas_measurement.items():
    i = 0
    for vp, rtt_list in measurements.items():
        if len(rtt_list) > 3:
            # only keep the minimum rtt
            filtered_measurements[target_ip][vp] = min(rtt_list)

    # check that we have measurements from more that one vp
    if len(measurements) < 3:
        filtered_measurements.pop(target_ip)
        unchecked_probes[target_ip] = measurements

    print(
        f"target ip: {target_ip}, nb_measurements: {len(filtered_measurements[target_ip])}")


print(
    f"atlas probes with passive measurements: {len(filtered_measurements)}, unchecked probes: {len(unchecked_probes)}")

107
45.138.229.91 : number of measurement with more than one packet: 15
77.174.76.85 : number of measurement with more than one packet: 1
82.217.219.124 : number of measurement with more than one packet: 0
83.81.82.33 : number of measurement with more than one packet: 0
193.0.0.78 : number of measurement with more than one packet: 284
79.12.239.224 : number of measurement with more than one packet: 0
77.174.30.45 : number of measurement with more than one packet: 0
93.108.63.51 : number of measurement with more than one packet: 0
86.89.224.211 : number of measurement with more than one packet: 0
216.147.121.123 : number of measurement with more than one packet: 0
76.82.152.84 : number of measurement with more than one packet: 0
2.57.252.147 : number of measurement with more than one packet: 1
81.220.141.64 : number of measurement with more than one packet: 0
85.225.8.64 : number of measurement with more than one packet: 0
88.163.164.208 : number of measurement with more than one packet

In [ ]:
# check if ips in results are within atlas probes set
for dst, measurements in atlas_measurement.items():
    for src, rtt_list in measurements.items():
        if src in probes_atlas.keys():
            src_latitude = probes_atlas[src][2]
            src_longitude = probes_atlas[src][3]

            dst_latitude = probes_atlas[dst][2]
            dst_longitude = probes_atlas[dst][3]

            dist = distance(
                lat1=float(src_latitude),
                lon1=float(src_longitude),
                lat2=float(dst_latitude),
                lon2=float(dst_longitude),
            )

            print(
                f"geoloc src: {src}=[{src_latitude}, {src_longitude}] | geoloc dst: {probes_atlas[dst][:]} |rtt = {min(rtt_list)} | dist = {dist}")

geoloc src: 46.183.219.225=[56.9515, 24.1115] | geoloc dst: [60052, False, 52.3685, 4.8995] |rtt = 35.99373 | dist = 1330.9468631458333
geoloc src: 185.114.152.90=[42.3485, -71.0615] | geoloc dst: [60052, False, 52.3685, 4.8995] |rtt = 95.445672 | dist = 5558.333526642797
geoloc src: 192.172.226.235=[32.8815, -117.2415] | geoloc dst: [60052, False, 52.3685, 4.8995] |rtt = 158.975832 | dist = 9002.09353065142
geoloc src: 213.192.184.42=[60.2015, 24.9295] | geoloc dst: [60052, False, 52.3685, 4.8995] |rtt = 33.994832 | dist = 1502.5986617113351
geoloc src: 197.239.73.59=[12.3715, -1.5085] | geoloc dst: [60052, False, 52.3685, 4.8995] |rtt = 112.793753 | dist = 4484.270724323151
geoloc src: 185.233.100.99=[44.8575, -0.5615] | geoloc dst: [60052, False, 52.3685, 4.8995] |rtt = 23.913838 | dist = 926.0231243763532
geoloc src: 196.49.14.25=[5.5975, -0.1885] | geoloc dst: [60052, False, 52.3685, 4.8995] |rtt = 119.303678 | dist = 5221.602307864207
geoloc src: 78.141.215.164=[52.3885, 4.6605] 

In [ ]:
measurement_file = "../datasets/measurement_atlas.pickle"

with open(measurement_file, "wb") as f:
    pickle.dump(atlas_measurement, f)

## verify probe location based on measurements

In [ ]:
# first method, simple, check if RTT coherent for all probes

# load everything (testing)
PROBES_FILE = Path(".") / "../datasets/probe_atlas.pickle"

with open(PROBES_FILE, "rb") as f:
    probes_atlas = pickle.load(f)


measurement_path = Path(".") / "../datasets/measurement_atlas.pickle"

with open(measurement_path, "rb") as f:
    measurement_atlas = pickle.load(f)

In [ ]:
from copy import deepcopy

filtered_measurements = deepcopy(atlas_measurement)
for dst, measurements in atlas_measurement.items():
    for src, rtt_list in measurements.items():
        if len(rtt_list) <= 1:
            # remove measurement where only one packet was sent
            filtered_measurements[dst].pop(src)

filtered_probes = deepcopy(filtered_measurements)
for dst, measurements in filtered_measurements.items():
    if len(measurements) <= 1:
        # remove destination if not enough measurements
        filtered_probes.pop(dst)

print(
    f"over all probes analyzed: {len(atlas_measurement)}, only {len(filtered_probes)} kept")

over all probes analyzed: 4, only 2 kept


## compare dist / rtt

In [ ]:
from analysis.helpers import rtt_to_km

percentage_dist_similarity = {}
for dst, measurements in filtered_probes.items():
    percentage_dist_similarity[dst] = []
    print(f"VALIDATION FOR PROBE : {dst}")
    for src, rtt_list in measurements.items():
        rtt_dist, dist = None, None
        if src in probes_atlas.keys():
            src_latitude = probes_atlas[src][2]
            src_longitude = probes_atlas[src][3]

            dst_latitude = probes_atlas[dst][2]
            dst_longitude = probes_atlas[dst][3]

            # get distance from lat-lon
            dist = distance(
                lat1=float(src_latitude),
                lon1=float(src_longitude),
                lat2=float(dst_latitude),
                lon2=float(dst_longitude),
            )

            # get distance from rtt
            rtt_dist = rtt_to_km(min(rtt_list))

            similarity = dist * 100 / rtt_dist
            percentage_dist_similarity[dst].append(similarity)

            print(
                f"src: {src}, rtt : {round(min(rtt_list),2)}, dist : {round(dist,2)}, dist (from rtt) : {round(rtt_dist,2)} ([{round(similarity)}% of similarity])")
    print(
        f"overall similarity (over {len(measurements)}) : {round(min(percentage_dist_similarity[dst]),2)}")
    print()

VALIDATION FOR PROBE : 45.138.229.91
src: 37.10.126.72, rtt : 144.52, dist : 8940.29, dist (from rtt) : 9634.64 ([93% of similarity])
src: 37.10.41.14, rtt : 240.05, dist : 10486.81, dist (from rtt) : 16003.25 ([66% of similarity])
src: 138.44.176.166, rtt : 341.02, dist : 14147.61, dist (from rtt) : 22734.35 ([62% of similarity])
overall similarity (over 22) : 73.52

VALIDATION FOR PROBE : 193.0.0.78
src: 192.134.1.25, rtt : 16.65, dist : 430.0, dist (from rtt) : 832.52 ([52% of similarity])
src: 193.170.114.242, rtt : 22.6, dist : 934.76, dist (from rtt) : 1129.98 ([83% of similarity])
src: 213.212.129.68, rtt : 23.77, dist : 835.23, dist (from rtt) : 1188.45 ([70% of similarity])
src: 213.128.137.33, rtt : 10.55, dist : 368.66, dist (from rtt) : 527.51 ([70% of similarity])
src: 130.59.80.2, rtt : 22.69, dist : 614.12, dist (from rtt) : 1134.47 ([54% of similarity])
src: 195.30.70.33, rtt : 14.7, dist : 666.25, dist (from rtt) : 735.22 ([91% of similarity])
src: 213.225.160.239, rtt